In [1]:
# Library import

import numpy as np
import math 

import tensorflow as tf

from keras import layers
from keras import models
from keras.layers import Dense, Dropout, Input
from keras.utils import to_categorical
from keras.metrics import MeanAbsoluteError
from keras.losses import CategoricalHinge, CategoricalCrossentropy
import keras.optimizers as Optimizers

import matplotlib.pyplot as plt

In [10]:
# Dataset Import

from keras.datasets import mnist


In [11]:
# Data Processing

(train_image, train_labels), (test_image, test_labels)= mnist.load_data();

train_image= train_image.reshape(60000, 28, 28, 1);
train_image= train_image.astype('float32')/255;

test_image= test_image.reshape(10000, 28, 28, 1);
test_image= test_image.astype('float32')/255;

train_labels= to_categorical(train_labels);
test_labels= to_categorical(test_labels);

In [12]:
# Check

print(np.shape(train_labels))
print(np.shape(test_labels))

(60000, 10)
(10000, 10)


In [13]:
# Dataset Sample Plot

#sample= 900
#image= train_image[sample]

#plt.figure(figsize= (5,5))
#plt.imshow(image, cmap= 'gray')
#print(train_labels[sample])

In [14]:
# Model Define - Feature Extractor

model= models.Sequential();
model.add(layers.Conv2D(64, (5,5), activation= 'relu', input_shape=(28, 28, 1)));
model.add(layers.MaxPooling2D(2,2));
model.add(layers.Conv2D(64, (3,3), activation= 'relu'));
model.add(layers.MaxPooling2D(2,2));
model.add(layers.Conv2D(32, (3,3), activation= 'relu'));

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 24, 24, 64)        1664      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 32)          18464     
                                                                 
Total params: 57,056
Trainable params: 57,056
Non-trai

In [15]:
# Classifier

model.add(layers.Flatten());
model.add(layers.Dense(64, activation= 'sigmoid'));
model.add(layers.Dense(10, activation= 'linear', name='svm'));


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 24, 24, 64)        1664      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 32)          18464     
                                                                 
 flatten_1 (Flatten)         (None, 288)              

In [16]:
# Model Train

model.compile(optimizer= 'adam',
             loss= CategoricalHinge(name='Categorical_Hinge'),
             metrics= ['accuracy']);

a=model.fit(train_image, train_labels, 
          epochs= 5,
          batch_size= 64,
          verbose= 1,
          validation_split= 0.2);

Epoch 1/5
750/750 [==============================] - 22s 29ms/step - loss: 0.1904 - accuracy: 0.9149 - val_loss: 0.0560 - val_accuracy: 0.9791
Epoch 2/5
750/750 [==============================] - 22s 29ms/step - loss: 0.0468 - accuracy: 0.9828 - val_loss: 0.0410 - val_accuracy: 0.9853
Epoch 3/5
750/750 [==============================] - 21s 28ms/step - loss: 0.0351 - accuracy: 0.9869 - val_loss: 0.0360 - val_accuracy: 0.9862
Epoch 4/5
750/750 [==============================] - 21s 28ms/step - loss: 0.0274 - accuracy: 0.9893 - val_loss: 0.0320 - val_accuracy: 0.9877
Epoch 5/5
750/750 [==============================] - 22s 29ms/step - loss: 0.0230 - accuracy: 0.9917 - val_loss: 0.0314 - val_accuracy: 0.9885


In [9]:
# Loss values
a_dict= a.history;
# print(a_dict.keys())

training_loss= a_dict['loss']
validation_loss= a_dict['val_loss']
train_accuracy= a_dict['accuracy']
validation_accuracy= a_dict['val_accuracy']

# Save data to .csv files
np.savetxt('train_loss.csv', training_loss, delimiter=',')
np.savetxt('validation_loss.csv', validation_loss, delimiter=',')

np.savetxt('train_accuracy.csv', train_accuracy, delimiter=',')
np.savetxt('validation_accuracy.csv', validation_accuracy, delimiter=',')


In [17]:
# Prediction

prediction = model.predict(test_image)

313/313 [==============================] - 3s 7ms/step


In [25]:
# Model Evauation

loss, accuracy= model.evaluate(test_image, test_labels);

print('Test loss:', loss)
print('Test accuracy:', accuracy* 100,'%')

313/313 [==============================] - 2s 6ms/step - loss: 0.0269 - accuracy: 0.9893
Test loss: 0.026883628219366074
Test accuracy: 98.9300012588501 %
